Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

pd.options.display.max_columns = 100

Загрузка обучающего и тестового датасетов

In [221]:
# train = pd.read_csv('data/train.csv')
# test = pd.read_csv('data/test.csv')

# def get_features(df: pd.DataFrame) -> pd.DataFrame:
    
#     # % полная сумма к уплате
#     df['psk'] = df['installment']*df['term']

#     # % переплаты
#     df['percent'] = (df['psk'] - df['funded_amnt'])/df['funded_amnt']

#     # годовой процент
#     df['percent_per_year'] = (df['percent']/df['term']*1200).round(decimals=1)

#     # ПДН без учета текущего займа
#     df['residual_dti'] = df['dti'] - (df['installment']*12/df['annual_inc'])*100
#     # отношение суммы займа к годовому доходу
#     df['loan_income_ratio'] = df['funded_amnt']/df['annual_inc']
    
#     # ПНД без учета займа
#     df['residual_dti'] = df['dti'] - (df['installment']*12/df['annual_inc'])*100
#     df['additional_dti'] = df['dti'] + (df['installment']*12/df['annual_inc'])*100
    
#     df['dti_ratio_min'] = df['dti']/df['residual_dti']
#     df['dti_ratio_max'] = df['dti']/df['additional_dti']
    
#     df['is_first_loan'] = df['mths_since_recent_inq'].between(6.01, 6.99).astype('int')
# #     df.loc[df['is_first_loan'], 'mths_since_recent_inq'] = None
#     df['is_not_sber_client'] = df['avg_cur_bal'].between(13841.01, 13841.02).astype('int')
# #     df.loc[
# #         df['is_not_sber_client'], ['num_accts_ever_120_pd', 'num_tl_90g_dpd_24m', 'avg_cur_bal', 'tot_hi_cred_lim']
# #     ] = None
    
#     return df

# train = get_features(train)
# test = get_features(test)

In [222]:
# Сумма платежа может быть неверной
# train = train[~(train['percent_per_year'] <= 0)].reset_index(drop=True)

In [223]:
# cols_to_scale = [
#     'installment',
#     'dti',
#     'funded_amnt',
#     'annual_inc',
#    'avg_cur_bal',
#    'tot_hi_cred_lim',
#    'psk',
#    'percent',
#    'percent_per_year',
#    'residual_dti',
#    'loan_income_ratio',
#    'additional_dti',
#    'dti_ratio_min',
#    'dti_ratio_max',
# #     'acc_open_past_24mths',
# ]
# scaller = StandardScaler()
# train[cols_to_scale] = scaller.fit_transform(train[cols_to_scale])
# test[cols_to_scale] = scaller.fit_transform(test[cols_to_scale])

In [224]:
# train

,installment,dti,funded_amnt,annual_inc,emp_length,term,inq_last_6mths,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,id,psk,percent,percent_per_year,residual_dti,loan_income_ratio,additional_dti,dti_ratio_min,dti_ratio_max,is_first_loan,is_not_sber_client
0,-0.337664,-0.589282,-0.506748,-0.151693,6.0,36,0,7.000000,0,0,0.0,0.0,4.0,-0.697848,-0.824599,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,-0.514251,-0.246725,0.165578,-0.376957,-0.587583,-0.647366,-0.002928,-0.035253,0,0
1,0.358838,0.213370,0.039209,0.139822,10.0,36,1,2.000000,1,0,3.0,0.0,4.0,-0.028681,-0.020516,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,-0.010540,-0.044758,0.570487,0.238100,-0.317610,0.151817,-0.007376,0.264248,0,0
2,0.283512,1.178109,0.937006,-0.503227,10.0,60,4,2.000000,0,0,0.0,0.0,3.0,-0.081573,-0.028197,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,0.956848,0.589176,-0.092091,0.633328,2.077148,1.392004,-0.005263,0.104535,0,0
3,-0.431637,-1.520307,-0.591675,-0.696141,0.0,36,1,4.000000,1,0,0.0,0.0,5.0,-0.758599,-0.809828,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,-0.582212,-0.213113,0.239198,-1.745748,0.156050,-1.041705,-0.049785,-2.137720,0,0
4,-0.272624,-1.631822,0.321287,0.011213,10.0,60,1,6.000000,0,0,0.0,0.0,3.0,1.375108,0.139633,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,0.286519,0.182152,-0.570620,-1.360593,0.110979,-1.535235,-0.082992,-1.593861,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,2.270642,-0.541304,1.859067,1.640270,10.0,36,0,2.000000,1,0,0.0,0.0,3.0,-0.626145,-0.354247,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,29995,1.372077,-0.570278,-0.460190,-0.511124,-0.202699,-0.460668,0.003626,-0.302866,0,0
29994,-0.303525,1.511359,0.039209,-0.288877,9.0,60,0,11.000000,0,0,0.0,0.0,2.0,-0.401307,-0.606181,0,0,0.0,0.0,1.0,0.0,0.0,

Разметка полей. def - таргет для модели. id - идентификатор. Все остальные поля - факторы для обучения модели

In [2]:
train = pd.read_csv('data/logreg_train.csv')
test = pd.read_csv('data/logreg_test.csv')

In [3]:
train

,installment,dti,funded_amnt,annual_inc,emp_length,term,inq_last_6mths,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,id,psk,percent,percent_per_year,residual_dti,loan_income_ratio,additional_dti,dti_ratio_min,dti_ratio_max,is_first_loan,is_not_sber_client
0,-0.337555,-0.589306,-0.506756,-0.151701,6.0,36,0,7.000000,0,0,0.0,0.0,4.0,-0.697867,-0.824611,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,-0.514169,-0.246028,0.165736,-0.377013,-0.587580,-0.647345,-0.002927,-0.035354,0,0
1,0.358919,0.213369,0.039218,0.139822,10.0,36,1,2.000000,1,0,3.0,0.0,4.0,-0.028682,-0.020503,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,-0.010466,-0.044315,0.569807,0.238039,-0.317605,0.151856,-0.007376,0.264127,0,0
2,0.283596,1.178135,0.937042,-0.503243,10.0,60,4,2.000000,0,0,0.0,0.0,3.0,-0.081575,-0.028184,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,0.956908,0.588823,-0.091400,0.633265,2.077170,1.392073,-0.005262,0.104425,0,0
3,-0.431525,-1.520356,-0.591685,-0.696162,0.0,36,1,4.000000,1,0,0.0,0.0,5.0,-0.758619,-0.809839,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,-0.582130,-0.212459,0.239204,-1.745793,0.156058,-1.041694,-0.049786,-2.137679,0,0
4,-0.272518,-1.631875,0.321305,0.011209,10.0,60,1,6.000000,0,0,0.0,0.0,3.0,1.375142,0.139651,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,0.286588,0.182311,-0.568938,-1.360641,0.110987,-1.535235,-0.082993,-1.593857,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2.270647,-0.541327,1.859131,1.640307,10.0,36,0,2.000000,1,0,0.0,0.0,3.0,-0.626161,-0.354244,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,29995,1.372130,-0.569175,-0.458737,-0.511178,-0.202693,-0.460643,0.003627,-0.302949,0,0
29996,-0.303418,1.511394,0.039218,-0.288888,9.0,60,0,11.000000,0,0,0.0,0.0,2.0,-0.401317,-0.606186,0,0,0.0,0.0,1.0,0.0,0.0,

In [4]:
targ_cols = [i for i in train.columns if i not in ['def', 'id']  and 'sub_grade' not in i]

X_train = train[targ_cols]
X_test = test[targ_cols]

y_train = train["def"]

Обучение модели

In [5]:
model = LogisticRegression(C=10, fit_intercept=True, max_iter=100,
          penalty='l1', random_state=123, solver = 'liblinear',
          tol=0.01).fit(X_train, y_train)


model.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=123, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [6]:
sorted(list(zip(model.coef_[0], targ_cols)))

[(-1.6227098227476482, 'home_ownership_NONE'),
 (-1.5397672300246126, 'purpose_vacation'),
 (-1.066424473326896, 'purpose_wedding'),
 (-0.8838482106160167, 'purpose_house'),
 (-0.7823375235574499, 'purpose_renewable_energy'),
 (-0.7497265169907722, 'purpose_car'),
 (-0.6313049395550484, 'purpose_credit_card'),
 (-0.6051854814468484, 'purpose_other'),
 (-0.5973651306357803, 'purpose_debt_consolidation'),
 (-0.49223224533700755, 'purpose_home_improvement'),
 (-0.48448736141086857, 'purpose_major_purchase'),
 (-0.4632817176620573, 'purpose_medical'),
 (-0.40304862652876083, 'purpose_moving'),
 (-0.2755456623346479, 'chargeoff_within_12_mths'),
 (-0.20107102971662616, 'home_ownership_OWN'),
 (-0.19566916713546864, 'installment'),
 (-0.17912950684202203, 'is_first_loan'),
 (-0.1729821110863801, 'home_ownership_MORTGAGE'),
 (-0.07197038255146458, 'tot_hi_cred_lim'),
 (-0.04675334408083442, 'avg_cur_bal'),
 (-0.044477091822750316, 'dti_ratio_max'),
 (-0.036930242679828806, 'additional_dti'),


In [10]:
lr_params = {
    "C": 1.5,
    "fit_intercept": True,
    "max_iter": 300,
    "penalty": "l1",
    "random_state": 123,
    "solver": "liblinear"
}

model = LogisticRegression(**lr_params)

cols = [
    'percent_per_year',
    'percent',
    'term',
    'loan_income_ratio',
    'psk',
    'additional_dti',
    'purpose_small_business',
    'purpose_home_improvement',
    'purpose_major_purchase',
    'purpose_medical',
    'acc_open_past_24mths',
    'num_accts_ever_120_pd',
    'home_ownership_MORTGAGE',
    'home_ownership_RENT',
    'purpose_vacation',
    'purpose_wedding',
    'purpose_car',
    'installment',
    'is_first_loan',
    'avg_cur_bal',

]

skf = StratifiedKFold(n_splits=5, random_state=16, shuffle=True)
scores = cross_val_score(model, X_train[cols], y_train, cv=skf, scoring='roc_auc')
print(np.mean(scores), np.std(scores))

0.6854250925856873 0.006137511823808159


In [19]:
def do_lr_cv(x_train, x_test, y_train, features, params):
    oof_result = pd.Series([0] * x_train.shape[0], index=x_train.index)
    result = [0] * x_test.shape[0]
    scores = []
    for ind_trn, ind_test in skf.split(x_train, y_train):
        X_train_df = x_train[features].iloc[ind_trn]
        X_valid_df = x_train[features].iloc[ind_test]
        y_train_df = y_train.iloc[ind_trn]
        y_valid_df = y_train.iloc[ind_test]

        model = LogisticRegression(**params)
        model.fit(X_train_df, y_train_df)

        scores.append(roc_auc_score(y_valid_df, model.predict_proba(X_valid_df)[:, 1]))
        result += model.predict_proba(x_test[features])[:, 1:].flatten()/5
        oof_result.loc[X_valid_df.index] = model.predict_proba(X_valid_df)[:, 1]
    return scores, result, oof_result

In [20]:
scores, result, oof_result = do_lr_cv(
    X_train,
    X_test,
    y_train,
    cols,
    lr_params,
)

In [24]:
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

(0.6854250925856873, 0.006137511823808159, 0.6853867901437658)

Рассчет метрики на обучающей модели

In [6]:
roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])

0.6858439828784051

Применение модели к тестовой выборке

In [28]:
test['prob'] = result

Выгрузка решения для коммита в соревновании

In [29]:
test[['id', 'prob']].to_csv('lr_submission.csv', index=False)

In [30]:
test[['id', 'prob']]

,id,prob
0,0,0.129688
1,1,0.141625
2,2,0.235351
3,3,0.161320
4,4,0.159157
...,...,...
29995,29995,0.119276
29996,29996,0.044804
29997,29997,0.278472
29998,29998,0.033987
